# Read xml workspace and fit using pyhf

In [1]:
import pyhf
import pyhf.readxml
import json

pyhf.set_backend(pyhf.tensor.numpy_backend(),pyhf.optimize.minuit_optimizer())

In [2]:
main_xml_path = "TRExFitter/minimal_example/RooStats/minimal_example.xml"
parent_folder = "TRExFitter/"
json_path     = "workspace_from_xml.json"

### translate the `xml` to python and dump it to a `json` file for later use

In [3]:
spec = pyhf.readxml.parse(main_xml_path, parent_folder, track_progress=False)

with open(json_path, "w") as f:
    json.dump(spec, f, sort_keys=True, indent=4)

### load spec from file again

In [4]:
with open(json_path, "r") as f:
    spec = json.load(f)

### create the relevant pyhf objects

In [5]:
workspace = pyhf.Workspace(spec)
pdf = workspace.model(measurement_name=None) # get default model if None
data = workspace.data(pdf, with_aux=True)

minuit = pyhf.optimizer._make_minuit(pyhf.utils.loglambdav,
                                     data,
                                     pdf,
                                     pdf.config.suggested_init(),
                                     pdf.config.suggested_bounds())

### perform an unconstrained fit

In [6]:
%%time
result = minuit.migrad()
#minuit.minos()

CPU times: user 57.5 ms, sys: 5.81 ms, total: 63.3 ms
Wall time: 58.7 ms


In [7]:
# get the names of all parameters
# this code is from https://github.com/scikit-hep/pyhf/pull/653
labels = ["{}[{}]".format(k, i) if pdf.config.param_set(k).n_parameters > 1 else k
          for k in pdf.config.par_order
          for i in range(pdf.config.param_set(k).n_parameters)]

bestfit = pyhf.tensorlib.astensor([x[1] for x in minuit.values.items()])
errors = pyhf.tensorlib.astensor([x[1] for x in minuit.errors.items()])

for i, l in enumerate(labels):
    print(f"{l}: {bestfit[i]:.3f} +/- {errors[i]:.3f}")

lumi: 1.002 +/- 0.020
staterror_Signal_region[0]: 1.000 +/- 0.006
staterror_Signal_region[1]: 0.999 +/- 0.006
staterror_Signal_region[2]: 1.003 +/- 0.005
staterror_Signal_region[3]: 0.998 +/- 0.006
staterror_Signal_region[4]: 0.999 +/- 0.008
staterror_Signal_region[5]: 0.997 +/- 0.012
Luminosity: 0.001 +/- 0.974
Signal_norm: 1.186 +/- 0.037


### show the correlation matrix

In [8]:
minuit.print_matrix()

-----------------------------------------------------------------------
|    |     p0     p1     p2     p3     p4     p5     p6     p7     p8 |
-----------------------------------------------------------------------
| p0 |  1.000 -0.181 -0.175 -0.023  0.034 -0.004 -0.040 -0.945 -0.234 |
| p1 | -0.181  1.000  0.323  0.043 -0.062  0.007  0.073 -0.008  0.432 |
| p2 | -0.175  0.323  1.000  0.056 -0.044  0.017  0.075 -0.007  0.398 |
| p3 | -0.023  0.043  0.056  1.000  0.240  0.155  0.068 -0.001 -0.256 |
| p4 |  0.034 -0.062 -0.044  0.240  1.000  0.173  0.052  0.001 -0.431 |
| p5 | -0.004  0.007  0.017  0.155  0.173  1.000  0.042 -0.000 -0.210 |
| p6 | -0.040  0.073  0.075  0.068  0.052  0.042  1.000 -0.002  0.013 |
| p7 | -0.945 -0.008 -0.007 -0.001  0.001 -0.000 -0.002  1.000 -0.010 |
| p8 | -0.234  0.432  0.398 -0.256 -0.431 -0.210  0.013 -0.010  1.000 |
-----------------------------------------------------------------------


parameters 0 and 7 (`lumi` and `Luminosity`) are highly anti-correlated